# Fault modeling example
Made for ESCAPE program - 2022



In [ ]:
# import statements - general statements
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize

# cartopy imports
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt

# these two are for plotting shapefiles
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature

# import statements - specific to our problem.
import fault_model
import fault_plots

# this special Jupyter command makes zooming, panning and downloading the figures easy
#%matplotlib widget

# import statement for an interactive plot
from ipywidgets import interactive


In [ ]:
# load GPS data
gpsdata = pd.read_csv('GPS_Ridgecrest.csv')

# I will convert them all to arrays at the start.
gpslon = np.asarray(gpsdata["X"])
gpslat = np.asarray(gpsdata["Y"])
gps_dE = np.asarray(gpsdata["E"])
gps_dN = np.asarray(gpsdata["N"])

In [ ]:
# create a cartopy map to view the GPS data
stamen_terrain = cimgt.Stamen('terrain-background')
fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator()) # if changing the projection, change it here only - the others should stay PlateCarree
ax.add_feature(cfeature.STATES, linestyle='-')
lonmin=-119
lonmax=-115.5
latmin=34.5
latmax=37
ax.set_extent([lonmin,lonmax,latmin,latmax])
ax.add_image(stamen_terrain,8) # number sets map resolution (higher is better, but slower)
ax.coastlines()
ax.gridlines(draw_labels=True) # Add gridlines and labels


# read an ArcGIS shapefile
# Add shapefile
ridgecrest_shpFile = './Surface_Rupture_Ridgecrest_Prov_Rel_1/Surface_Rupture_Ridgecrest_Prov_Rel_1.shp'
ridgecrest_feature = ShapelyFeature(Reader(ridgecrest_shpFile).geometries(), ccrs.PlateCarree(), facecolor='none')
ax.add_feature(ridgecrest_feature,edgecolor='orange', linewidth=1) # Plot surface rupture feature

# Plot vectors
vecscale = 140
q = ax.quiver(gpslon,gpslat,gps_dE,gps_dN, scale=vecscale, transform=ccrs.PlateCarree())
plt.quiverkey(q,X=0.77,Y=0.93,U=10, label ='10 cm') # make quiver legend


# Making inset
# Making the inset here
left, bottom, width, height = [0.72, 0.62, 0.2, 0.2]
ax2 = fig.add_axes([left, bottom, width, height],projection=ccrs.PlateCarree())
ax2.set_extent([-125, -114, 50, 30], crs=ccrs.PlateCarree())
ax2.add_feature(cfeature.COASTLINE)
ax2.add_feature(cfeature.STATES)
# Including a box to show location
plt.plot([lonmin,lonmax,lonmax,lonmin,lonmin],[latmin,latmin,latmax,latmax,latmin],'-r')
plt.plot(-117.5,35.6,'*',color='orange')

# uncomment this line to save map as a pdf
# plt.savefig(my_map.pdf)

plt.show()

In [ ]:
# cartopy is too slow. Let's create a simpler plot with just the vectors

fig = plt.figure(figsize = (7,7))
ax = fig.add_subplot(1, 1, 1)

# note that for a non-geographic plot, we use set_xlim and set_ylim instead of set_extent.
ax.set_xlim([-119, -115.5])
ax.set_ylim([34.5, 37])
ax.grid() # grid instead of gridlines

# commands for quiver are the same, except for 'transform' option
q = ax.quiver(gpslon,gpslat,gps_dE,gps_dN, scale=vecscale)
ax.quiverkey(q,X=0.85,Y=0.85,U=10, label ='10 cm') # make quiver legend


In [ ]:
# define a function that implements the fault modeling functions defined in fault_model.py.
def create_fault_model(gpslon,gpslat,latcorner,loncorner,depthcorner,strike,dip,L,W,ss,ds):
    Fmod=fault_model.FaultModel()
    Fmod.create_planar_model(latcorner=latcorner,loncorner=loncorner,depthcorner=depthcorner,strike=strike,dip=dip,L=L,W=W,nL=1,nW=1)

    # get the "G" matrix for our particular GPS site locations (lat,lon) - note this is Y,X in our notation, which can be confusing.
    G=Fmod.get_greens(gpslat,gpslon)
    
    # m is the slip
    m=np.array([ss,ds])
    
    # predict GPS displacements with d=G*m
    gpsmotion = np.matmul(G,m)
    predicted_E = gpsmotion[0::3] # this notation means start from first element (0), then take every 3rd one
    predicted_N = gpsmotion[1::3]
    predicted_U = gpsmotion[2::3]
    
    return predicted_E,predicted_N,predicted_U,Fmod

# define a function for all the plotting and mapping commands, including running the fault model
# the equals signs in the first line (function definition) sets up a default value for each parameter
def plot_fault_and_gps(latcorner=35.5,loncorner=-117.3,depthcorner=0,strike=-35,dip=85,L=100e3,W=20e3,ss=-100,ds=0):
    # create a fault model
    # note, we pass gpslon and gpslat here, but it is not an input to this function. 
    # we are assuming it is already a global variable...
    predicted_E,predicted_N,predicted_U,Fmod = create_fault_model(gpslon,gpslat,latcorner,loncorner,depthcorner,strike,dip,L,W,ss,ds)

    # plot the fault model and GPS
    Fplot = fault_plots.FaultPlot2D(figsize=(10,10))
    Fplot.plot_symbols(gpslon,gpslat,'g^',markersize=3,label='GPS sites')
    Fplot.plot_slip_patches(Fmodel=Fmod,slip=[ss])

    # get the axes object so we can do normal things to it
    ax = Fplot.get_ax()
    ax.set_xlim([-119, -115.5])
    ax.set_ylim([34.5, 37])
    ax.set_title('Depth in meters')
    ax.legend()

    # commands for quiver are the same
    q = ax.quiver(gpslon,gpslat,gps_dE,gps_dN, scale=vecscale)
    q2= ax.quiver(gpslon,gpslat,predicted_E,predicted_N, scale=vecscale,color='red')
    ax.quiverkey(q,X=0.85,Y=0.8,U=10, label ='10 cm') # make quiver legend
    ax.quiverkey(q2,X=0.85,Y=0.7,U=10, label ='model') # make quiver legend


#run the model & plot command - now we can pass just some arguments
plot_fault_and_gps(ss=-150)

In [ ]:
# why did we put all the plotting commands inside an annoying function? Because now we can make it interactive!
w = interactive(plot_fault_and_gps,latcorner=(35,37,.01),loncorner=(-118,-116.5,.01),depthcorner=(0,30e3),strike=(-180,180),dip=(0,180),L=(0,200e3),W=(0,200e3),ss=(-300,300),ds=(-100,100))
display(w)

In [ ]:
# let's use scipy.optimize.curve_fit to fit the GPS data.
# to use this, we need a function that takes as input the independent variables (station locations) as a single vector, and model parameters (fault),
# and outputs the dependent variables (gps displacements) as a single vector.

# note there is a numerical instability for this case. We fix the fault width to make things work better. Try changing it and see what happens!
fix_width = 10000 # 10km default

def fault_model_for_fitting(gps_locs,latcorner,loncorner,depthcorner,strike,dip,L,ss,ds):
    # for the modeling, we need the locations as x and y, not a single vector
    Ngps=int(np.size(gps_locs)/2)
    gpslat=gps_locs[:Ngps]
    gpslat=gps_locs[Ngps:]
    
    # create the model
    Fmod=fault_model.FaultModel()
    Fmod.create_planar_model(latcorner=latcorner,loncorner=loncorner,depthcorner=depthcorner,strike=strike,dip=dip,L=L,W=fix_width,nL=1,nW=1)
    
    # get the "G" matrix for our particular GPS site locations (lat,lon) - note this is Y,X in our notation, which can be confusing.
    G=Fmod.get_greens(gpslat,gpslon)
    
    # m is the slip
    m=np.array([ss,ds])
    
    # predict GPS displacements with d=G*m
    gpsmotion = np.matmul(G,m)
    predicted_E = gpsmotion[0::3] # this notation means start from first element (0), then take every 3rd one
    predicted_N = gpsmotion[1::3]
    
    # get the displacements as a single vector
    predicted_displacements=np.append(predicted_E,predicted_N)
    
    # return the predicted values
    return predicted_displacements

# create single vectors for the locations and displacements
gps_locations=np.append(gpslon,gpslat)
gps_displacements=np.append(gps_dE,gps_dN)

m,mcov = scipy.optimize.curve_fit(fault_model_for_fitting,gps_locations,gps_displacements,p0=[35.5,-117.3,0,-35,85,100e3,-100,0],maxfev=10000)
print(m)


In [ ]:

# get the predicted GPS displacements, by running the fault model again with the best fitting parameters (m)
gps_predicted=fault_model_for_fitting(gps_locations,m[0],m[1],m[2],m[3],m[4],m[5],m[6],m[7]) 

#split the displacements back into E and N
Ngps=int(np.size(gps_predicted)/2)
predicted_E=gps_predicted[:Ngps]
predicted_N=gps_predicted[Ngps:]

# create a plot with results
fig = plt.figure(figsize = (7,7))
ax = fig.add_subplot(1, 1, 1)

# note that for a non-geographic plot, we use set_xlim and set_ylim instead of set_extent.
ax.set_xlim([-119, -115.5])
ax.set_ylim([34.5, 37])
ax.grid() # grid instead of gridlines

# commands for quiver are the same
q = ax.quiver(gpslon,gpslat,gps_dE,gps_dN, scale=vecscale)
q2 = ax.quiver(gpslon,gpslat,predicted_E,predicted_N, scale=vecscale,color='red')
ax.quiverkey(q,X=0.85,Y=0.85,U=10, label ='10 cm') # make quiver legend
ax.quiverkey(q2,X=0.85,Y=0.75,U=10, label ='model') # make quiver legend

print('Best fitting solution: lat %f, lon %f, depth %f, strike %f, dip %f, length %f, ss %f, ds %f' %(m[0],m[1],m[2],m[3],m[4],m[5],m[6],m[7]))


In [ ]:
# what is the magnitude?
length = m[5]
width = fix_width
slip = np.sqrt(m[6]**2+m[7]**2)/100 # units must be in meters, not cm

import moment_tensor
mag = moment_tensor.get_magnitude(length,fix_width,slip)
print(mag)